# This is  a research notebook to try to find correlation between hyperwave and volume 

In [125]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)


In [126]:
from datetime import datetime
import json
import requests

from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

In [127]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [128]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [129]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [130]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'quandl',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Netflix= {
    'name':'Netflix',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}

data_Abiomed = {
    'name':'Abiomed',
    'data_source':'investopedia',
    'symbol':'ABMD',
}

data_Alphabet = {
    'name':'Alphabet',
    'data_source':'investopedia',
    'symbol':'GOOGL',
}

data_BTCUSD = {
    'name':'BTCUSD',
    'data_source':'CryptoCompare',
    'symbol':'BTC-USD',
}



In [131]:
company_setup = data_BTCUSD
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 1)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [140]:

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class CryptoCompare_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-SUN').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        from_symbol, to_symbol  = self._symbol.split('-')
        url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&allData=true&aggregate=3&e=CCCAGG".format( \
                                                                                                                        from_symbol, \
                                                                                                                        to_symbol)

        r = requests.get(url)
        array = json.dumps(r.json())
        
        data = json.loads(array)
        daily_tickers = pd.DataFrame(data["Data"])
        daily_tickers['date'] = pd.to_datetime(df['time'],unit='s')
        daily_tickers = daily_tickers.rename( columns={"volumeto": "volume"})
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

    
# https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG
    
sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
    "CryptoCompare": CryptoCompare_Loader
}

def get_historical_data(symbol, source, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    dir(source_class)
    return source_class.get_dataframe()
    

In [137]:
# url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG"
# df = pd.read_json(url)
# df = pd.DataFrame(df.locations.values.tolist())['Data']
# display(df)

import json
import pandas as pd
import requests
r = requests.get(url)
# print r.json() #
array = json.dumps(r.json())
# print(data["Data"])
data = json.loads(array)
df = pd.DataFrame(data["Data"])
df['date'] = pd.to_datetime(df['time'],unit='s')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,close,high,low,open,time,volumefrom,volumeto,date
0,0.08080,0.09307,0.04951,0.04951,1279324800,669.01,5.574000e+01,2010-07-17
1,0.05050,0.08181,0.05050,0.08080,1279584000,2997.00,1.926300e+02,2010-07-20
2,0.05050,0.06767,0.05049,0.05050,1279843200,4450.30,2.528600e+02,2010-07-23
3,0.05890,0.06200,0.05000,0.05050,1280102400,8640.98,4.995900e+02,2010-07-26
4,0.06785,0.06990,0.05600,0.05890,1280361600,15101.59,9.707500e+02,2010-07-29
5,0.06000,0.06500,0.05900,0.06785,1280620800,16021.46,9.899000e+02,2010-08-01
6,0.06230,0.06240,0.05700,0.06000,1280880000,9923.07,6.002900e+02,2010-08-04
7,0.07100,0.07350,0.05900,0.06230,1281139200,18451.09,1.176870e+03,2010-08-07
8,0.07000,0.07541,0.06000,0.07100,1281398400,17433.88,1.239000e+03,2010-08-10
9,0.06529,0.06950,0.06450,0.07000,1281657600,12458.84,8.240600e+02,2010-08-13


## Function that allow to calculate the weekid from a start date

In [134]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["date"]-base_date).days/7)

def clean_raw_market_data(df):
#     df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
#     df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    return df.sort_values(by="date")

def get_weekId_max_price(df):
    return df.loc[ df.loc[:,('close')].argmax()]['weekId']


In [135]:
def get_weekId_cartesian_product(df):
    df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
    df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [142]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_historical_data(symbol, data_source, 'Weekly')

df_weekly_price = clean_raw_market_data(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)
df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

In [143]:
print(max_price_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price_raw )

11372


,open,close,high,low,volume,date,is_price_closing_up,weekId
date,,,,,,,,
2010-07-19,0.04951,0.08080,0.09307,0.04951,5.574000e+01,2010-07-19,False,10985
2010-07-26,0.05050,0.05050,0.08181,0.05000,9.450800e+02,2010-07-26,False,10986
2010-08-02,0.06785,0.06000,0.06990,0.05600,1.960650e+03,2010-08-02,True,10987
2010-08-09,0.06230,0.07100,0.07350,0.05700,1.777160e+03,2010-08-09,True,10988
2010-08-16,0.06529,0.06529,0.07690,0.06000,3.878690e+03,2010-08-16,False,10989
2010-08-23,0.06640,0.06500,0.06790,0.06120,2.925140e+03,2010-08-23,False,10990
2010-08-30,0.06500,0.06497,0.06900,0.03211,3.853460e+03,2010-08-30,False,10991
2010-09-06,0.06160,0.06160,0.06490,0.05961,3.065630e+03,2010-09-06,False,10992
2010-09-13,0.06366,0.06199,0.17500,0.06014,2.458620e+03,2010-09-13,True,10993


In [144]:
# print(max_price_weekId)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

In [145]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

In [146]:
def get_mean_square_error(y_true, y_pred):
    y_square_diff = np.square(y_true-y_pred)
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
    lower_item = y_true[y_true <= y_pred]
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

def get_hull_tangent(df_input):
    hull = ConvexHull(df_input[['weekId', 'close']].dropna())
    
    hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
    data_from_to = [{"x1": df_input['weekId'].iloc[pair[0]], \
       "y1": df_input['close'].iloc[pair[0]], \
       "x2": df_input['weekId'].iloc[pair[1]], \
       "y2": df_input['close'].iloc[pair[1]]} for pair in hull_results]
    df = pd.DataFrame(data_from_to)
    df = get_line_slope_and_origine(df)
    df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
    df['days'] = np.abs(df['x1'] - df['x2'])
    df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_input),axis=1)
    df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_input),axis=1)
    df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
    df['ratio_slope_y1'] = df['m']/df['y1']
    df['ratio_slope_y2'] = df['m']/df['y2']
    return df

def get_upper_path(df):
    row = df.iloc[0]
    upper_path = [row['x1'], row['x2']]

    while(row['x2'] != max_price_weekId):
        row = df.loc[df['x1'] == row['x2']].head().iloc[0]
        upper_path.append(row['x2'])

    return upper_path

def delete_path(df, path):
    df_cleaned = df.copy(True)
    
    for x1, x2 in zip(path[:-1], path[1:]):
        df_cleaned = df_cleaned[(np.logical_not((df_cleaned.x1 == x1) & (df_cleaned.x2 == x2))) ]

    return df_cleaned.sort_values(['x1','y2'], ascending=[True, False])

def delete_upper_path(df):
    return delete_path(df, get_upper_path(df))

In [147]:
# hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
# data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
#    "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
#    "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
#    "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
# df = pd.DataFrame(data_from_to)

In [148]:
df_hull_tangent = get_hull_tangent(df_weekly_price_until_max)
display(df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False]))
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# df_hull_tangent = delete_upper_path(df_hull_tangent)

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,10985,11372,0.0808,18972.3200,49.023874,-5.385272e+05,88.831430,387,1.008982e+08,388,2.600468e+05,606.731113,0.002584
1,10985,10986,0.0808,0.0505,-0.030300,3.329263e+02,-1.735531,1,3.755759e+06,2,1.877879e+06,-0.375000,-0.600000
19,10986,10996,0.0505,0.0614,0.001090,-1.192424e+01,0.062452,10,3.741042e+06,5,7.482085e+05,0.021584,0.017752
23,10996,11007,0.0614,0.2401,0.016245,-1.785736e+02,0.930714,11,3.734183e+06,12,3.111819e+05,0.264584,0.067661
22,11007,11011,0.2401,0.3130,0.018225,-2.003625e+02,1.044100,4,3.733319e+06,12,3.111099e+05,0.075906,0.058227
21,11011,11022,0.3130,0.7790,0.042364,-4.661530e+02,2.425807,11,3.722943e+06,8,4.653679e+05,0.135347,0.054382
20,11022,11055,0.7790,2.2000,0.043061,-4.738350e+02,2.465668,33,3.722655e+06,8,4.653318e+05,0.055277,0.019573
12,11055,11082,2.2000,5.1400,0.108889,-1.201567e+03,6.214390,27,3.698555e+06,5,7.397109e+05,0.049495,0.021185
17,11082,11084,5.1400,5.5700,0.215000,-2.377490e+03,12.133880,2,3.664065e+06,7,5.234379e+05,0.041829,0.038600
18,11084,11106,5.5700,10.8700,0.240909,-2.664666e+03,13.544973,22,3.655788e+06,10,3.655788e+05,0.043251,0.022163


In [149]:
df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False])

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,10985,11372,0.0808,18972.3200,49.023874,-5.385272e+05,88.831430,387,1.008982e+08,388,2.600468e+05,606.731113,0.002584
1,10985,10986,0.0808,0.0505,-0.030300,3.329263e+02,-1.735531,1,3.755759e+06,2,1.877879e+06,-0.375000,-0.600000
19,10986,10996,0.0505,0.0614,0.001090,-1.192424e+01,0.062452,10,3.741042e+06,5,7.482085e+05,0.021584,0.017752
23,10996,11007,0.0614,0.2401,0.016245,-1.785736e+02,0.930714,11,3.734183e+06,12,3.111819e+05,0.264584,0.067661
22,11007,11011,0.2401,0.3130,0.018225,-2.003625e+02,1.044100,4,3.733319e+06,12,3.111099e+05,0.075906,0.058227
21,11011,11022,0.3130,0.7790,0.042364,-4.661530e+02,2.425807,11,3.722943e+06,8,4.653679e+05,0.135347,0.054382
20,11022,11055,0.7790,2.2000,0.043061,-4.738350e+02,2.465668,33,3.722655e+06,8,4.653318e+05,0.055277,0.019573
12,11055,11082,2.2000,5.1400,0.108889,-1.201567e+03,6.214390,27,3.698555e+06,5,7.397109e+05,0.049495,0.021185
17,11082,11084,5.1400,5.5700,0.215000,-2.377490e+03,12.133880,2,3.664065e+06,7,5.234379e+05,0.041829,0.038600
18,11084,11106,5.5700,10.8700,0.240909,-2.664666e+03,13.544973,22,3.655788e+06,10,3.655788e+05,0.043251,0.022163


In [150]:
print('Valeant : {}', 2.99/257.53)
print('netflix : {}', 7.065833/411.09)
print('amazon : {}', 17.596667/ 2012.71)

Valeant : {} 0.011610297829379104
netflix : {} 0.017188043980636843
amazon : {} 0.008742773176463573


In [151]:
def graph_hyperwave(df_data, df_hyperwave):

    x = df_data['weekId'].values
    y =  df_data['close'].values
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = 'lines'
    )

    layout = {
    #     'xaxis': {
    #         'range': [0, 7]
    #     },
    #     'yaxis': {
    #         'range': [0, 2.5]
    #     },
        'shapes': [{
                'type': 'line',
                'x0': row[1]['x1'],
                'y0': row[1]['y1'],
                'x1': row[1]['x2'],
                'y1': row[1]['y2'],
                'line': {
                    'color': 'rgb(55, 128, 191)',
                    'width': 3,
                }
            } for row in df_hyperwave.iterrows()
            # Line Vertical
            ]
    }
    data = [trace0]

    # py.iplot(data, filename='line-mode')
    fig = {
        'data': data,
        'layout': layout,
    }

    return fig
#     py.iplot(fig, filename='shapes-lines')

In [152]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price_until_max)
# df =delete_upper_path(df_weekly_price_until_max)
# display(df)

In [153]:
df = df_weekly_price_until_max
max_weekId = 10899
nb_tail_items = 104

def get_max_price(df, column_name = 'close'):
    return df.loc[ df.loc[:,(column_name)].argmax()][column_name]
# weekId_max_price = get_weekId_max_price(last_n_weeks_Items) 
#     max_price_last_n_weeks = last_n_weeks_Items[(last_n_weeks_Items.weekId == weekId_max_price )].iloc[0]

last_n_weeks_Items = df[(df.weekId < max_weekId)].tail(nb_tail_items)    
max_price = get_max_price(last_n_weeks_Items )

print( max_price)

df_week_greater_than = df[(df.weekId >= max_weekId)]

# df.loc[ df.loc[:,('close')].argmax()]['weekId']
df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
weekId_start_hyperwave = df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']
df_week_greater_than = df[(df.weekId >= weekId_start_hyperwave )]
df_hyperwave = get_hull_tangent(df_week_greater_than)
display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))

py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# py.iplot(fig, filename='shapes-lines')

ValueError: attempt to get argmax of an empty sequence

In [ ]:

# display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))
# print(max_price_weekId)
# print(get_upper_path(df_hyperwave))
display(delete_upper_path(df_hyperwave, get_upper_path(df_hyperwave)))

In [ ]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)


# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)

# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
